## Start

In [ ]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

["id",                                                                                           
 "issue_id", - это на будущее,  когда будет группировка жалоб по ишьб                                                                                    
 "src_type", какая сущность пожаловалсь (Citizen/Camera/Intercom)                                                                             
 "src_uuid", - uuid этой сущности                                                                                 
 "src_address_uuid", - адрес этой сущности                                                                      
 "src_address_type",     адрес этой сущности                                                                        
 "device_uuid", - uuid устройтсва на которое жалуется сущность                                                                              
 "device_type",                                                                
 "err_code", код ошибки intercom_offline / video / no_intercom                                                                           
 "err_src", откуда пришло MOB (мобилка) / WEB (веб интерфейсы для партнеров) / API (бэк )/ DEV (устройство)                                                                              
 "err_func",  sync (там еще будет)                                                                                   
 "aasm_state",  - статус жалобы pending / processed / failed                                                                             
 "process_error", - ошибка если статус failed                                                                           
 "actor_type", - какой живой человек с сердцем пожаловался или авто система (будет либо Citizen, Partner, auto)
 "actor_identifier", - uuid / id актера
 "created_at",
 "updated_at"]

In [ ]:
# creating a table from s3

query_text = """--sql 
CREATE TABLE db1.complaints_st_partner
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code`	String,
    `err_src` String,	
    `err_func`	String,
    `aasm_state` String,	
    `process_error`	String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime,
    `updated_at` DateTime,
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/complaints_st_partner/year=*/month=*/*.csv', 'CSVWithNames')
PARTITION BY report_date
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE TABLE db1.complaints_st_partner_ch
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code`	String,
    `err_src` String,	
    `err_func`	String,
    `aasm_state` String,	
    `process_error`	String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime,
    `updated_at` DateTime,
)
    ENGINE = MergeTree()
    ORDER BY src_uuid
    """

ch.query_run(query_text)

In [12]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.complaints_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR RANDOMIZE FOR 1 HOUR TO db1.complaints_st_partner_ch AS
    SELECT
        *
    FROM db1.complaints_st_partner
ORDER BY report_date DESC
"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [13]:
query_text = """--sql
    SELECT
        *
    FROM db1.complaints_st_partner_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


report_date,issue_id,src_type,src_uuid,src_address_uuid,src_address_type,device_uuid,device_type,err_code,err_src,err_func,aasm_state,process_error,actor_type,actor_identifier,created_at,updated_at
date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[ns],datetime[ns]
2025-08-03,"""""","""intercom""","""a858c9bd-2bf0-486a-95dd-847c2f…","""cd341ca4-ad20-4147-8ef3-f74b90…","""entry""","""a858c9bd-2bf0-486a-95dd-847c2f…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 05:00:12,2025-08-03 05:00:13
2025-08-03,"""""","""intercom""","""a88ef229-8877-4d2c-a93f-77a6fd…","""""","""""","""a88ef229-8877-4d2c-a93f-77a6fd…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 02:30:36,2025-08-03 02:30:36
2025-08-03,"""""","""intercom""","""a913744c-519b-4093-947d-d443e0…","""a6bee4db-2bdf-48dc-9f38-8960e6…","""entry""","""a913744c-519b-4093-947d-d443e0…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 22:30:23,2025-08-03 22:30:24
2025-08-03,"""""","""intercom""","""a9188197-475a-4957-9de7-d10133…","""99fff647-d4d5-49d1-80d5-cfec47…","""entry""","""a9188197-475a-4957-9de7-d10133…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 00:00:16,2025-08-03 00:00:17
2025-08-03,"""""","""Citizen""","""a96dfed0-83d3-457e-935d-e4e573…","""173fec4e-44e8-46da-9fc7-114480…","""entry""","""""","""""","""no_intercom""","""MOB""","""sync""","""processed""","""""","""""","""""",2025-08-03 18:31:58,2025-08-03 18:31:59
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-08-03,"""""","""intercom""","""c1a6e108-3651-457f-b769-a37b41…","""f9ddfac3-e5ed-4b4d-b6ec-48648d…","""entry""","""c1a6e108-3651-457f-b769-a37b41…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 05:30:04,2025-08-03 05:30:05
2025-08-03,"""""","""intercom""","""c05af074-3569-4cbd-a14a-9aa5dc…","""469df41e-1d2d-444b-b6a7-489c30…","""entry""","""c05af074-3569-4cbd-a14a-9aa5dc…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 11:00:15,2025-08-03 11:00:17
2025-08-03,"""""","""intercom""","""c096bc5a-bde3-4b17-b7e9-f09d53…","""f5e9b85f-2f15-4f47-9257-1f0e79…","""entry""","""c096bc5a-bde3-4b17-b7e9-f09d53…","""intercom""","""intercom_offline""","""API""","""sync""","""processed""","""""","""auto""","""api""",2025-08-03 09:00:31,2025-08-03 09:00:32


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.complaints_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner_mv
    """

ch.query_run(query_text)

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.complaints_st_partner_mv
"""

ch.query_run(query_text)